In [1]:
import polartoolkit
from polartoolkit import fetch, maps, profiles, regions

In [2]:
import pathlib
import pooch
from tqdm import tqdm
import requests
import pandas as pd

In [3]:
import logging


logger = logging.getLogger(__name__)

import verde as vd
import invert4geom
import numpy as np
import pandas as pd
import harmonica
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from polartoolkit import fetch, maps, profiles, regions

import os

os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

region = regions.getz_ice_shelf

In [4]:
url_list = ['https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.10.30/IGGRV1B_20101030_11190700_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.10.30/IGGRV1B_20101030_11190700_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.05/IGGRV1B_20101105_11512800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.05/IGGRV1B_20101105_11512800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.31/IGGRV1B_20161031_12281500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.31/IGGRV1B_20161031_12281500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.13/IGGRV1B_20121013_12580800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.13/IGGRV1B_20121013_12580800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.15/IGGRV1B_20181115_11523800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.15/IGGRV1B_20181115_11523800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.02/IGGRV1B_20141102_10162900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.02/IGGRV1B_20141102_10162900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.27/IGGRV1B_20121027_11230800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.27/IGGRV1B_20121027_11230800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.11/IGGRV1B_20111111_11433200_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.11/IGGRV1B_20111111_11433200_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.03/IGGRV1B_20161103_12092900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.03/IGGRV1B_20161103_12092900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.05/IGGRV1B_20161105_12014000_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.05/IGGRV1B_20161105_12014000_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.16/IGGRV1B_20091016_11013000_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.16/IGGRV1B_20091016_11013000_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.09/IGGRV1B_20111109_10554400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.09/IGGRV1B_20111109_10554400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.09/IGGRV1B_20161109_11444900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.09/IGGRV1B_20161109_11444900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.02/IGGRV1B_20161102_12211900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.02/IGGRV1B_20161102_12211900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.07/IGGRV1B_20161107_11532100_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.07/IGGRV1B_20161107_11532100_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.23/IGGRV1B_20121023_13062800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.23/IGGRV1B_20121023_13062800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.21/IGGRV1B_20141121_10163100_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.21/IGGRV1B_20141121_10163100_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.28/IGGRV1B_20091028_11240400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.28/IGGRV1B_20091028_11240400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.11/IGGRV1B_20181111_12085800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.11/IGGRV1B_20181111_12085800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.20/IGGRV1B_20161020_12355800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.20/IGGRV1B_20161020_12355800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.12/IGGRV1B_20091112_12344500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.12/IGGRV1B_20091112_12344500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.03/IGGRV1B_20141103_10253900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.03/IGGRV1B_20141103_10253900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.10.17/IGGRV1B_20111017_10331900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.10.17/IGGRV1B_20111017_10331900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.04/IGGRV1B_20111104_13313800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.04/IGGRV1B_20111104_13313800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.13/IGGRV1B_20111113_11351100_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.13/IGGRV1B_20111113_11351100_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.15/IGGRV1B_20161115_11195400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.11.15/IGGRV1B_20161115_11195400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.07/IGGRV1B_20141107_10125100_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.07/IGGRV1B_20141107_10125100_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.28/IGGRV1B_20161028_12182900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.28/IGGRV1B_20161028_12182900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.03/IGGRV1B_20111103_11000800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.03/IGGRV1B_20111103_11000800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.12/IGGRV1B_20121012_11344800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2012.10.12/IGGRV1B_20121012_11344800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.10.31/IGGRV1B_20181031_13465700_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.10.31/IGGRV1B_20181031_13465700_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.03/IGGRV1B_20181103_11233800_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2018.11.03/IGGRV1B_20181103_11233800_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2013.11.25/IGGRV1B_20131125_17123900_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2013.11.25/IGGRV1B_20131125_17123900_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.13/IGGRV1B_20141113_10431700_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.13/IGGRV1B_20141113_10431700_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.06/IGGRV1B_20101106_11465700_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.06/IGGRV1B_20101106_11465700_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.18/IGGRV1B_20091118_11060500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.18/IGGRV1B_20091118_11060500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.19/IGGRV1B_20101119_16531400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.19/IGGRV1B_20101119_16531400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.10.23/IGGRV1B_20111023_10540400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.10.23/IGGRV1B_20111023_10540400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.12/IGGRV1B_20111112_12105300_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2011.11.12/IGGRV1B_20111112_12105300_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.21/IGGRV1B_20091021_12014500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.21/IGGRV1B_20091021_12014500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.02/IGGRV1B_20091102_11082400_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.11.02/IGGRV1B_20091102_11082400_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.18/IGGRV1B_20091018_11011500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2009.10.18/IGGRV1B_20091018_11011500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.11/IGGRV1B_20141111_10150500_V019.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2014.11.11/IGGRV1B_20141111_10150500_V019.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.16/IGGRV1B_20091016_11515650_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.16/IGGRV1B_20091016_11515650_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.16/IGGRV1B_20091016_11515650_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.18/IGGRV1B_20091018_11442050_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.18/IGGRV1B_20091018_11442050_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.18/IGGRV1B_20091018_11442050_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.21/IGGRV1B_20091021_12433200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.21/IGGRV1B_20091021_12433200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.21/IGGRV1B_20091021_12433200_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.28/IGGRV1B_20091028_11592500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.28/IGGRV1B_20091028_11592500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.10.28/IGGRV1B_20091028_11592500_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.02/IGGRV1B_20091102_13064950_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.02/IGGRV1B_20091102_13064950_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.02/IGGRV1B_20091102_13064950_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.12/IGGRV1B_20091112_13580700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.12/IGGRV1B_20091112_13580700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.12/IGGRV1B_20091112_13580700_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.18/IGGRV1B_20091118_11492450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.18/IGGRV1B_20091118_11492450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2009.11.18/IGGRV1B_20091118_11492450_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.10.30/IGGRV1B_20101030_11531750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2010.10.30/IGGRV1B_20101030_11531750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.10.30/IGGRV1B_20101030_11531750_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.05/IGGRV1B_20101105_12532350_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.05/IGGRV1B_20101105_12532350_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.05/IGGRV1B_20101105_12532350_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.06/IGGRV1B_20101106_12554650_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.06/IGGRV1B_20101106_12554650_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.06/IGGRV1B_20101106_12554650_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.19/IGGRV1B_20101119_17290550_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.19/IGGRV1B_20101119_17290550_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2010.11.19/IGGRV1B_20101119_17290550_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.17/IGGRV1B_20111017_11494300_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.17/IGGRV1B_20111017_11494300_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.17/IGGRV1B_20111017_11494300_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.23/IGGRV1B_20111023_11491000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.23/IGGRV1B_20111023_11491000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.10.23/IGGRV1B_20111023_11491000_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.03/IGGRV1B_20111103_11464750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.03/IGGRV1B_20111103_11464750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.03/IGGRV1B_20111103_11464750_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.04/IGGRV1B_20111104_14314700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.04/IGGRV1B_20111104_14314700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.04/IGGRV1B_20111104_14314700_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.09/IGGRV1B_20111109_12031600_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.09/IGGRV1B_20111109_12031600_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.09/IGGRV1B_20111109_12031600_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.11/IGGRV1B_20111111_13592250_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.11/IGGRV1B_20111111_13592250_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.11/IGGRV1B_20111111_13592250_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.12/IGGRV1B_20111112_13481200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.12/IGGRV1B_20111112_13481200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.12/IGGRV1B_20111112_13481200_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.13/IGGRV1B_20111113_12430500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.13/IGGRV1B_20111113_12430500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2011.11.13/IGGRV1B_20111113_12430500_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.12/IGGRV1B_20121012_11550450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.12/IGGRV1B_20121012_11550450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.12/IGGRV1B_20121012_11550450_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.13/IGGRV1B_20121013_13172000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.13/IGGRV1B_20121013_13172000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.13/IGGRV1B_20121013_13172000_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.23/IGGRV1B_20121023_13262900_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.23/IGGRV1B_20121023_13262900_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.23/IGGRV1B_20121023_13262900_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.27/IGGRV1B_20121027_11433100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.27/IGGRV1B_20121027_11433100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2012.10.27/IGGRV1B_20121027_11433100_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2013.11.25/IGGRV1B_20131125_18560050_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2013.11.25/IGGRV1B_20131125_18560050_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2013.11.25/IGGRV1B_20131125_18560050_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.02/IGGRV1B_20141102_11423800_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.02/IGGRV1B_20141102_11423800_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.02/IGGRV1B_20141102_11423800_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.03/IGGRV1B_20141103_11431000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.03/IGGRV1B_20141103_11431000_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.03/IGGRV1B_20141103_11431000_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.07/IGGRV1B_20141107_11393500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.07/IGGRV1B_20141107_11393500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.07/IGGRV1B_20141107_11393500_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.11/IGGRV1B_20141111_11455100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.11/IGGRV1B_20141111_11455100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.11/IGGRV1B_20141111_11455100_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.13/IGGRV1B_20141113_11463900_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.13/IGGRV1B_20141113_11463900_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.13/IGGRV1B_20141113_11463900_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.21/IGGRV1B_20141121_11454150_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.21/IGGRV1B_20141121_11454150_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2014.11.21/IGGRV1B_20141121_11454150_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.20/IGGRV1B_20161020_12504700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.20/IGGRV1B_20161020_12504700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.20/IGGRV1B_20161020_12504700_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.26/IGGRV1B_20161026_12495550_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.26/IGGRV1B_20161026_12495550_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.26/IGGRV1B_20161026_12495550_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.28/IGGRV1B_20161028_12535500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.28/IGGRV1B_20161028_12535500_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.28/IGGRV1B_20161028_12535500_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.31/IGGRV1B_20161031_13030750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.31/IGGRV1B_20161031_13030750_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.10.31/IGGRV1B_20161031_13030750_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.02/IGGRV1B_20161102_12471700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.02/IGGRV1B_20161102_12471700_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.02/IGGRV1B_20161102_12471700_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.03/IGGRV1B_20161103_13115450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.03/IGGRV1B_20161103_13115450_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.03/IGGRV1B_20161103_13115450_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.05/IGGRV1B_20161105_12472350_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.05/IGGRV1B_20161105_12472350_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.05/IGGRV1B_20161105_12472350_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.07/IGGRV1B_20161107_12445850_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.07/IGGRV1B_20161107_12445850_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.07/IGGRV1B_20161107_12445850_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.09/IGGRV1B_20161109_12445200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.09/IGGRV1B_20161109_12445200_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.09/IGGRV1B_20161109_12445200_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.15/IGGRV1B_20161115_12473400_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.15/IGGRV1B_20161115_12473400_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2016.11.15/IGGRV1B_20161115_12473400_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.10.31/IGGRV1B_20181031_14545400_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2018.10.31/IGGRV1B_20181031_14545400_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.10.31/IGGRV1B_20181031_14545400_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.11/IGGRV1B_20181111_12491100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.11/IGGRV1B_20181111_12491100_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.11/IGGRV1B_20181111_12491100_V020.txt.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.15/IGGRV1B_20181115_12395800_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/opendap/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.15/IGGRV1B_20181115_12395800_V020.txt',
 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.002/2018.11.15/IGGRV1B_20181115_12395800_V020.txt.xml']

In [5]:
def fetch_oib_gravity():
    """
    Fetch OIB gravity data from the NSIDC and merge into 1 file.

    """
    count=0

    if pathlib.Path(f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv").exists():
        df = pd.read_csv(
            f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv",
            # index=False,
        )
    else:
        urls = [t for t in url_list if t.endswith("txt")]

        dfs = []
        for url in tqdm(urls, total=len(urls), desc="txt files"):
            try:
                fname = pooch.retrieve(
                    url=url,
                    path=f"{pooch.os_cache('pooch')}/goat/gravity",
                    # fname="bedmap2_point_data.zip",
                    known_hash=None,
                    progressbar=True,
                    # processor=preprocessing,
                    downloader=fetch.EarthDataDownloader(),
                )
            except requests.exceptions.HTTPError as e:
                logger.error(e)
                logger.error(f"Failed to download {url}")
                count+=1
                continue
            df = pd.read_csv(
                fname,
                comment="#",
                header=None,
                usecols=[5,6,7,8,14,15,16,17],
                names=[
                    'line',
                    'x',
                    'y',
                    'height',
                    "levelling_correction",
                    'free_air_gravity_70s',
                    'free_air_gravity_100s',
                    'free_air_gravity_140s',
                ],
                sep=r"\s+",
            )
            df = df.dropna()
            df = utils.points_inside_region(df, getz_region)
            dfs.append(df)

        df = pd.concat(dfs)
        df.to_csv(f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv", index=False)

    if count>0:
        logger.warning(f"Failed to download {count} files")

    return df.rename(columns={"x": "easting", "y": "northing"})

In [6]:
fetch_oib_gravity()

txt files:   0%|          | 0/129 [00:00<?, ?it/s]Downloading data from 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.10.30/IGGRV1B_20101030_11190700_V019.txt' to file 'C:\Users\iii\AppData\Local\pooch\pooch\Cache\goat\gravity\be41a8eb31d8b92d6e1ff0417aca480a-IGGRV1B_20101030_11190700_V019.txt'.


Enter your Earthdata Login username:  a_b_c_d
Enter your Earthdata password:  ········


404 Client Error: Not Found for url: https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.10.30/IGGRV1B_20101030_11190700_V019.txt
Failed to download https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.10.30/IGGRV1B_20101030_11190700_V019.txt
txt files:   1%|          | 1/129 [01:42<3:38:43, 102.53s/it]Downloading data from 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.05/IGGRV1B_20101105_11512800_V019.txt' to file 'C:\Users\iii\AppData\Local\pooch\pooch\Cache\goat\gravity\d71c4695418813e84d1d1d9c6633b3d2-IGGRV1B_20101105_11512800_V019.txt'.
404 Client Error: Not Found for url: https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.05/IGGRV1B_20101105_11512800_V019.txt
Failed to download https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2010.11.05/IGGRV1B_20101105_11512800_V019.txt
txt files:   2%|▏         | 2/129 [01:45<1:33:07, 43.99s/it] Downloading data from 'https://n5eil01u.ecs.nsidc.org/DP4/ICEBRIDGE/IGGRV1B.001/2016.10.31

ValueError: No objects to concatenate